In [1]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn import preprocessing as pp
from matplotlib.font_manager import FontProperties

%matplotlib inline
myfont=FontProperties(fname='/usr/share/fonts/truetype/yahei.ttf',size=14)
mpl.rcParams['axes.unicode_minus']=False
sns.set(font=myfont.get_name())

In [3]:
# 数据读取
dateparse = lambda dates: pd.datetime.strptime((str(dates)+'0101')[0:8], '%Y%m%d') if not pd.isnull(dates) else None
company_df = pd.read_csv('/home/phoenixkiller/notebook/bond_data/company_info.csv',
                         parse_dates=['成立日期'],
                         infer_datetime_format = True,
                         date_parser=dateparse)
bond_df = pd.read_csv('/home/phoenixkiller/notebook/bond_data/a.csv',
                      parse_dates=['报告期'],
                      infer_datetime_format = True,
                      date_parser=dateparse)
level_df = pd.read_csv('/home/phoenixkiller/notebook/bond_data/level.csv',
                       parse_dates=['评级日期','公告日期'],
                       infer_datetime_format=True,
                       date_parser=dateparse)
# 标记财务报表类型(年报，半年报，季报)
bond_df['报告类型']=bond_df['报告期'].apply(lambda x: '年报' if x.month==12 else ('半年报' if x.month==6 else '季报'))

# 关联公司基本信息及评级信息
company = pd.merge(company_df, level_df, how='inner', left_on='公司名称', right_on='公司中文名称')

# 去掉不适用的评级
company = company[~company['信用评级'].isin(['A1','A2','A3','Baa1','Ba1','Aa3','B2','Baa3','Ba3','B1'])]

# 删除无用列
company = company.drop(['公司中文简介', '信用评级说明', '法人代表', '总经理', '董事会秘书', '主页', '电子邮箱',
                        '经营范围', '对象ID', '公司ID', '公司中文名称', '主要产品及业务', '办公地址',
                        '评级展望', '债券主体公司id', '前次信用评级', '评级变动方向', '城市'], axis=1)

# 设置列类型
company[['公司类别','评级类型','评级机构代码']] = company[['公司类别','评级类型','评级机构代码']].astype(str)
company['员工总数人'] = company[company['员工总数人'].notnull()]['员工总数人'].astype(int)


In [4]:
bond_fault_df = pd.read_csv('/home/phoenixkiller/notebook/bond_data/bond_fault.csv',
                         parse_dates=['发生日期'],
                         infer_datetime_format = True,
                         date_parser=dateparse)


In [8]:
bond_fault_df['发行人'].shape


(148,)

In [ ]:
思路：
增加发行时主体评级，以及当前信用评级，将信用评级数值化，并计算信用评级差
可以关联同一家公司的多个违约记录，以便增加样本数量
关联舆情分类指标（一个月，一个季度，半年，一年）统计各时间段内舆情分类发生次数